In [1]:
import warnings

warnings.filterwarnings("ignore")
import os
import glob
from datetime import datetime
import io
import pandas as pd
import numpy as np

import statsmodels.api as sm
import logging

# 로그 설정
logging.basicConfig(filename='error.log', level=logging.ERROR)

from itertools import product
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib import font_manager, rc
import seaborn as sns


In [25]:

# data_h_df = pd.read_csv('./temp_data.csv', index_col='dt', parse_dates=['dt'])


def sarima_learn(data_h_df):
    p, q = range(0, 2), range(0, 2)
    d = range(0, 1)
    P, Q = range(0, 2), range(0, 2)
    D = range(0, 1)
    m = 24
    trend_pdq = list(product(p, d, q))
    seasonal_pdq = [
        (candi[0], candi[1], candi[2], m) for candi in list(product(P, D, Q))
    ]

    # all_list = data_h_df.TABLE_INDEX.unique()
    # ld_nm_list=all_list[:]
    # ld_nm_list=all_list[5:15]

    file_list = []

    # print('부하명 : ', ld_nm)
    ## 최소학습
    # train_df = data_h_df['DIS'][len(data_h_df[data_h_df['TABLE_INDEX'] == ld_nm]) - (24*60):len(data_h_df[data_h_df['TABLE_INDEX'] == ld_nm]) - (24*30)]
    # test_df = data_h_df['DIS'][len(data_h_df[data_h_df['TABLE_INDEX'] == ld_nm]) - (24*30):]
    cut_idx = round(len(data_h_df["val"]) * 0.8)

    # train_df = data_h_df["val"][:cut_idx]
    # test_df = data_h_df["val"][cut_idx:]
    train_df = data_h_df[:cut_idx]
    test_df = data_h_df[cut_idx:]

    
    train_df = train_df.asfreq("H", method="ffill")
    test_df = test_df.asfreq("H", method="ffill")
    Y_train_feR = train_df.copy()
    Y_test_feR = test_df.copy()
    Y_train_feR = Y_train_feR + 0.00000000000000000001
    Y_test_feR = Y_test_feR + 0.00000000000000000001

    try:
        ##  AUTO SARIMAX (최적파라미터)
        AIC = []
        print("11111111111111")
        SARIMAX_order = []
        for trend_param in tqdm(trend_pdq):
            for seasonal_params in seasonal_pdq:
                try:
                    result = sm.tsa.SARIMAX(
                        Y_train_feR,
                        trend="c",
                        order=trend_param,
                        seasonal_order=seasonal_params,
                    ).fit()
                    print(
                        "Fit SARIMAX: trend_order={} seasonal_order={} AIC={}, BIC={}".format(
                            trend_param,
                            seasonal_params,
                            result.aic,
                            result.bic,
                            end="\r",
                        )
                    )
                    AIC.append(result.aic)
                    SARIMAX_order.append([trend_param, seasonal_params])
                except Exception as e:
                    # 모든 예외에 대한 처리
                    print("예외 발생: %s", str(e))
                    continue
        ## Parameter Selection
        print(
            "The smallest AIC is {} for model SARIMAX{}x{}".format(
                min(AIC),
                SARIMAX_order[AIC.index(min(AIC))][0],
                SARIMAX_order[AIC.index(min(AIC))][1],
            )
        )
        print("222222222222222")
        fit_ts_sarimax = sm.tsa.SARIMAX(
            Y_train_feR,
            trend="c",
            order=SARIMAX_order[AIC.index(min(AIC))][0],
            seasonal_order=SARIMAX_order[AIC.index(min(AIC))][1],
        ).fit()

        ## Prediction
        print("33333333333333")
        pred_tr_ts_sarimax = fit_ts_sarimax.predict()
        pred_te_ts_sarimax = fit_ts_sarimax.get_forecast(
            steps=len(Y_test_feR)
        ).predicted_mean
        pred_te_ts_sarimax_ci = fit_ts_sarimax.get_forecast(
            steps=len(Y_test_feR)
        ).conf_int()

        def forecast_one_step():
            pred = fit_ts_sarimax.get_forecast(1)
            return pred.predicted_mean, pred.conf_int()
        print("4444444444444444")
        result_df = pd.DataFrame()
        res = pd.DataFrame()
        for values in Y_test_feR.val.values:
            pred, ci = forecast_one_step()
            result_df = pd.concat([pd.DataFrame({"predict": pred}), ci], axis=1)
            res = pd.concat([res, result_df])
            fit_ts_sarimax = fit_ts_sarimax.extend(np.array([values]))
        print("5555555555555555")
        val_df = pd.concat([pd.DataFrame(res.predict), Y_test_feR], axis=1)
        val_df["dif"] = (val_df.predict - val_df.val).abs()
        val_df["dif/val"] = (val_df["dif"] / val_df["val"] * 100).abs()
        print("666666666666666")
        file_list.append(
            [
                "test",
                SARIMAX_order[AIC.index(min(AIC))][0],
                SARIMAX_order[AIC.index(min(AIC))][1],
                100 - val_df["dif/val"].mean(),
            ]
        )
        fit_ts_sarimax.save("./ld_sarima_" + ".pkl")
        print("7777777777777")
        one_list = []
        one_list.append(
            [
                "test",
                SARIMAX_order[AIC.index(min(AIC))][0],
                SARIMAX_order[AIC.index(min(AIC))][1],
                100 - val_df["dif/val"].mean(),
            ]
        )
        with open("./parameter.csv", "a") as f:
            for param in one_list:
                print(str(param))
                f.writelines("%s\n" % str(param))
        one_list = []

    except Exception as e:
        # 모든 예외에 대한 처리
        print("예외 발생: %s", str(e))
        file_list.append(["test", "예외발생"])


# sarima_learn(data_h_df)


In [26]:
df = pd.read_csv("./script/temp_data2.csv", index_col="dt", parse_dates=["dt"])
sarima_learn(df)

11111111111111


  0%|          | 0/4 [00:00<?, ?it/s]

Fit SARIMAX: trend_order=(0, 0, 0) seasonal_order=(0, 0, 0, 24) AIC=832.1277867516543, BIC=837.9234663515562
Fit SARIMAX: trend_order=(0, 0, 0) seasonal_order=(0, 0, 1, 24) AIC=773.197373738514, BIC=781.8908931383667
Fit SARIMAX: trend_order=(0, 0, 0) seasonal_order=(1, 0, 0, 24) AIC=724.4952058132903, BIC=733.188725213143


c:\Users\kdn\anaconda3\envs\ai\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
 25%|██▌       | 1/4 [00:01<00:03,  1.15s/it]

Fit SARIMAX: trend_order=(0, 0, 0) seasonal_order=(1, 0, 1, 24) AIC=713.6078370748448, BIC=725.1991962746484
Fit SARIMAX: trend_order=(0, 0, 1) seasonal_order=(0, 0, 0, 24) AIC=742.9489129934516, BIC=751.6424323933043
Fit SARIMAX: trend_order=(0, 0, 1) seasonal_order=(0, 0, 1, 24) AIC=709.8369390952826, BIC=721.4282982950862
Fit SARIMAX: trend_order=(0, 0, 1) seasonal_order=(1, 0, 0, 24) AIC=684.3780183497308, BIC=695.9693775495344


...

Fit SARIMAX: trend_order=(1, 0, 0) seasonal_order=(1, 0, 1, 24) AIC=641.4596365641701, BIC=655.9488355639246
Fit SARIMAX: trend_order=(1, 0, 1) seasonal_order=(0, 0, 0, 24) AIC=661.0646234560223, BIC=672.6559826558259
Fit SARIMAX: trend_order=(1, 0, 1) seasonal_order=(0, 0, 1, 24) AIC=656.6382921922913, BIC=671.1274911920458
Fit SARIMAX: trend_order=(1, 0, 1) seasonal_order=(1, 0, 0, 24) AIC=652.0080346917593, BIC=666.4972336915139


c:\Users\kdn\anaconda3\envs\ai\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
100%|██████████| 4/4 [00:05<00:00,  1.31s/it]


Fit SARIMAX: trend_order=(1, 0, 1) seasonal_order=(1, 0, 1, 24) AIC=639.679782590086, BIC=657.0668213897915
The smallest AIC is 639.679782590086 for model SARIMAX(1, 0, 1)x(1, 0, 1, 24)
222222222222222


c:\Users\kdn\anaconda3\envs\ai\lib\site-packages\statsmodels\base\model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


33333333333333
4444444444444444
5555555555555555
666666666666666
7777777777777
['test', (1, 0, 1), (1, 0, 1, 24), 92.78913748845136]


In [11]:
import threading

def list_threads():
    for thread in threading.enumerate():
        print(f"Thread name: {thread.name}")

if __name__ == "__main__":
    list_threads()

Thread name: MainThread
Thread name: IOPub
Thread name: Heartbeat
Thread name: Control
Thread name: IPythonHistorySavingThread
Thread name: Thread-4


In [63]:
# # 예측값(pred_value)과 실제값(real_value) 예시 데이터
# dates = [datetime(2023, 9, 12, 8), datetime(2023, 9, 12, 9), datetime(2023, 9, 12, 10), datetime(2023, 9, 12, 11), datetime(2023, 9, 12, 12),  ]
# pred_value = [10, 15, 12, 18, 20]
# real_value = [8, 14, 10, 16, 19]

def upload_to_today(filename):
    today = datetime.now().strftime("%Y-%m-%d")
    return f"media/images/{today}/{filename[:-4]}"

folder_name = upload_to_today('data9.csv')
# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [57]:
li = [
    {
        "id": 1,
        "pred_value": "null",
        "real_value": 9,
        "pred_dt": "2023-09-19T08:00:00+09:00",
        "created_dt": "2023-09-12T11:26:08.471608+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 2,
        "pred_value": "null",
        "real_value": 11,
        "pred_dt": "2023-09-19T09:00:00+09:00",
        "created_dt": "2023-09-12T11:26:22.867959+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 3,
        "pred_value": "null",
        "real_value": 14,
        "pred_dt": "2023-09-19T10:00:00+09:00",
        "created_dt": "2023-09-12T11:26:48.166535+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 4,
        "pred_value": "null",
        "real_value": 9,
        "pred_dt": "2023-09-19T11:00:00+09:00",
        "created_dt": "2023-09-12T11:26:58.507158+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 5,
        "pred_value": "null",
        "real_value": 8,
        "pred_dt": "2023-09-19T12:00:00+09:00",
        "created_dt": "2023-09-12T11:27:07.337118+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 6,
        "pred_value": "null",
        "real_value": 7,
        "pred_dt": "2023-09-19T13:00:00+09:00",
        "created_dt": "2023-09-19T15:42:20.629445+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 7,
        "pred_value": "null",
        "real_value": 10,
        "pred_dt": "2023-09-19T14:00:00+09:00",
        "created_dt": "2023-09-19T15:43:10.528932+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 8,
        "pred_value": "null",
        "real_value": 14,
        "pred_dt": "2023-09-19T15:00:00+09:00",
        "created_dt": "2023-09-19T15:43:39.661732+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 9,
        "pred_value": "null",
        "real_value": 12,
        "pred_dt": "2023-09-19T16:00:00+09:00",
        "created_dt": "2023-09-19T15:43:56.810963+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 10,
        "pred_value": 9,
        "real_value": 9,
        "pred_dt": "2023-09-19T17:00:00+09:00",
        "created_dt": "2023-09-19T15:44:18.982883+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 12,
        "pred_value": 10,
        "real_value": "null",
        "pred_dt": "2023-09-19T18:00:00+09:00",
        "created_dt": "2023-09-19T15:44:18.982883+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 13,
        "pred_value": 8,
        "real_value": "null",
        "pred_dt": "2023-09-19T19:00:00+09:00",
        "created_dt": "2023-09-19T15:44:18.982883+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 14,
        "pred_value": 7,
        "real_value": "null",
        "pred_dt": "2023-09-19T20:00:00+09:00",
        "created_dt": "2023-09-19T15:44:18.982883+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 15,
        "pred_value": 9,
        "real_value": "null",
        "pred_dt": "2023-09-19T21:00:00+09:00",
        "created_dt": "2023-09-19T15:44:18.982883+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 16,
        "pred_value": 11,
        "real_value": "null",
        "pred_dt": "2023-09-19T22:00:00+09:00",
        "created_dt": "2023-09-19T15:44:18.982883+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 17,
        "pred_value": 14,
        "real_value": "null",
        "pred_dt": "2023-09-19T23:00:00+09:00",
        "created_dt": "2023-09-19T15:44:18.982883+09:00",
        "file": 13,
        "model": 1
    },
    {
        "id": 18,
        "pred_value": 11,
        "real_value": "null",
        "pred_dt": "2023-09-20T00:00:00+09:00",
        "created_dt": "2023-09-19T15:44:18.982883+09:00",
        "file": 13,
        "model": 1
    }
]

In [58]:
# 데이터 변환
dates = []
pred_value = []
real_value = []

for item in li:
    # "pred_dt"에서 시간 정보 추출
    dt = datetime.fromisoformat(item["pred_dt"][:-6])  # 시간대 정보 삭제
    dates.append(dt)
    pred_value.append(item["pred_value"])
    real_value.append(item["real_value"])
    # pred_value.append(item["pred_value"]) if item["pred_value"] is not "null" else pred_value.append(0)
    # real_value.append(item["real_value"]) if item["real_value"] is not "null" else real_value.append(0)

# 결과 출력
print("dates =", dates)
print("pred_value =", pred_value)
print("real_value =", real_value)

dates = [datetime.datetime(2023, 9, 19, 8, 0), datetime.datetime(2023, 9, 19, 9, 0), datetime.datetime(2023, 9, 19, 10, 0), datetime.datetime(2023, 9, 19, 11, 0), datetime.datetime(2023, 9, 19, 12, 0), datetime.datetime(2023, 9, 19, 13, 0), datetime.datetime(2023, 9, 19, 14, 0), datetime.datetime(2023, 9, 19, 15, 0), datetime.datetime(2023, 9, 19, 16, 0), datetime.datetime(2023, 9, 19, 17, 0), datetime.datetime(2023, 9, 19, 18, 0), datetime.datetime(2023, 9, 19, 19, 0), datetime.datetime(2023, 9, 19, 20, 0), datetime.datetime(2023, 9, 19, 21, 0), datetime.datetime(2023, 9, 19, 22, 0), datetime.datetime(2023, 9, 19, 23, 0), datetime.datetime(2023, 9, 20, 0, 0)]
pred_value = ['null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 'null', 9, 10, 8, 7, 9, 11, 14, 11]
real_value = [9, 11, 14, 9, 8, 7, 10, 14, 12, 9, 'null', 'null', 'null', 'null', 'null', 'null', 'null']


In [59]:
dates2 = dates.copy()
pred_value2 = pred_value.copy()
real_value2 = real_value.copy()

In [60]:
pred_value = [float(val) if val != 'null' else float('nan') for val in pred_value]
real_value = [float(val) if val != 'null' else float('nan') for val in real_value]

In [61]:
real_value2 = [x for x in real_value2 if x != 'null']
pred_value2 = [x for x in pred_value2 if x != 'null']


In [1]:
# 산점도 그래프
def savefig_scatter():
    sns.set(style='whitegrid')
    plt.figure(figsize=(10, 6))

    sns.scatterplot(x=dates, y=real_value, label='Actual', color='red', marker='x', s=100)
    sns.scatterplot(x=dates, y=pred_value, label='Prediction', color='blue', marker='o', s=100)

    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title('Scatter Plot of Prediction vs. Actual')

    plt.legend()
    plt.xticks(rotation=45)  # Rotate x-axis labels

    plt.savefig(folder_name + '/scatter.png')
    plt.close()

# 히스토그램 그래프
def savefig_histogram():
    plt.figure(figsize=(10, 6))

    sns.histplot(real_value2, label='Actual', color='red', kde=True)
    sns.histplot(pred_value2, label='Prediction', color='blue', kde=True)

    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Prediction vs. Actual')

    plt.legend()

    plt.savefig(folder_name + '/histogram.png')
    plt.close()

# 박스 플롯 그래프
def savefig_boxplot():
    plt.figure(figsize=(10, 6))

    sns.boxplot(data=[real_value2, pred_value2], palette=['red', 'blue'])
    
    plt.ylabel('Value')
    plt.title('Box Plot of Prediction vs. Actual')

    plt.xticks([0, 1], ['Actual', 'Prediction'])
    plt.savefig(folder_name + '/boxplot.png')
    plt.close()

# 바이올린 플롯 그래프
def savefig_violinplot():
    plt.figure(figsize=(10, 6))

    sns.violinplot(data=[real_value2, pred_value2], palette=['red', 'blue'])
    
    plt.ylabel('Value')
    plt.title('Violin Plot of Prediction vs. Actual')

    plt.xticks([0, 1], ['Actual', 'Prediction'])

    plt.savefig(folder_name + '/violinplot.png')
    plt.close()

# 각 함수를 호출하여 그래프를 생성하고 저장
savefig_scatter()
savefig_histogram()
savefig_boxplot()
savefig_violinplot()

NameError: name 'sns' is not defined